# Lecture 8
## Linear Optimization with JuMP (Part 3)
## Date: 16.11

### Recap exercises

### Ex1

Suppose a consumer can consume two types of discrete (therefore not continuous) foods, one healthy in which the payoff is one, whereas the second food (unhealthy) returns a payoff of two per unit. The price of the two goods are respectively two and one and the consumer has a budget of ten money to spend on those. Given the higher payoff of the unhealthy food as well as the cheap price, the social planner imposed a tax that impacts on consumer's utility such that when the good two is consumed there is a *una tantum fee* to pay of 10.

Model the problem using JuMP and find the optimal bundle, moreover argue on how much the tax imposed should be to discourage consumption of the unhealthy food 2.

#### Solution

In [1]:
using JuMP, GLPK

# Preparing an optimization model
m = Model(with_optimizer(GLPK.Optimizer))

# Declaring variables
@variable(m, x1 >=0, Int)
@variable(m, x2 >=0, Int)
@variable(m, x3, Bin)

# Setting the objective
@objective(m, Max, x1 + 2x2 - 10x3)
    #+ 5x3)

# Adding constraints
@constraint(m, constraint1,  2x1 + x2 <= 10)
@constraint(m, constraint2,  x2 <= 10*x3)

# Printing the prepared optimization model
print(m)

# Solving the optimization problem
JuMP.optimize!(m)

# Printing the optimal solutions obtained
println("Optimal Solutions:")
println("x1 = ", JuMP.value(x1))
println("x2 = ", JuMP.value(x2))
println("x3 = ", JuMP.value(x3))

Max x1 + 2 x2 - 10 x3
Subject to
 x3 binary
 x1 integer
 x2 integer
 x1 ≥ 0.0
 x2 ≥ 0.0
 2 x1 + x2 ≤ 10.0
 x2 - 10 x3 ≤ 0.0
Optimal Solutions:
x1 = 0.0
x2 = 10.0
x3 = 1.0


### Ex2

Suppose there is a freight train with limited capacity of say ten tons. This train has enough fuel to pursue only one trip and because of the limited capacity cannot carry all the five goods, that has respectively a weight of: two, eight, four, two, five. This goods provides also different profits to the to the sender for an amount respectively: five, three, two, seven, four.

Solve the problem using JuMP, moreover think of a case in which the train is allowed to pursue more than one trip and the sender wants to know the optimal schedule for each trip.

#### Solution

In [2]:
profit = [5, 3, 2, 7, 4]
weight = [2, 8, 4, 2, 5]
    
capacity = 10

model = Model(with_optimizer(GLPK.Optimizer))
    
@variable(model, x[1:5], Bin)
    
# Objective: maximize profit
@objective(model, Max, profit' * x)

# Constraint: can carry all
@constraint(model, weight' * x <= capacity)

# Solve problem using MIP solver
JuMP.optimize!(model)

# Printing the optimal solutions obtained
println("Optimal Solutions:")
println("x = ", JuMP.value.(x))

Optimal Solutions:
x = [1.0, 0.0, 0.0, 1.0, 1.0]


### Dual problem

Confronted with an optimization problem involving scarce resources, an economist will often ask: **What happens to the optimal solution if the availability of the resources changes?**

For linear programming problems, answers to questions like this are intimately related to the so-called **Duality Theory** of Linear Problems.

#### Example

Take the baker problem we solved graphically in the first lecture on Linear Programming.

$$
\text{Maximize } 20x_1 + 30x_2 \text{ s.t. } \begin{cases} 3x_1 + 6x_2 \leq 150 \\ x_1 + 0.5 x_2 \leq 22 \\ x_1 + x_2 \leq 27.5 \\ x_1, x_2 \in \mathbb{R}_+ \end{cases}
$$

**Exercise:** try to translate the problem into JuMP (try to do it in a form that is as compact as possible).

In [3]:
profit = [20, 30];

cake_A_costs = [3,1,1];
cake_B_costs = [6,0.5,1];

boundaries = [150,22,27.5];

In [4]:
model = Model(with_optimizer(GLPK.Optimizer));

In [5]:
@variable(model, x[1:2] >= 0, base_name = "Cake");

In [6]:
@objective(model, Max, profit' * x);

In [7]:
@constraint(model, con, (x[1].* cake_A_costs) + (x[2].* cake_B_costs) .<= boundaries); 

3-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 3 Cake[1] + 6 Cake[2] ≤ 150.0
 Cake[1] + 0.5 Cake[2] ≤ 22.0 
 Cake[1] + Cake[2] ≤ 27.5     

In [8]:
model

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: con, x

In [9]:
JuMP.optimize!(model)

# Printing the optimal solutions obtained
println("Optimal Solutions:")
println("x = ", JuMP.value.(x))

Optimal Solutions:
x = [5.0, 22.5]


#### Back to the example 

Now suppose the baker wants to become a **franchisee** and sell the ingredients to an **entrant** who will take the business, therefore the **entrant** will like to **minimize its expenditures** provided that the **profits for the franchisee** will stay the same as in the **primal problem**. 

This kind of problem is called **dual**. 

#### Exercise

Formulate the dual problem

##### Solution

In [10]:
model_dual = Model(with_optimizer(GLPK.Optimizer));

In [11]:
@variable(model_dual, u[1:3] >= 0, base_name = "Price");

In [12]:
@objective(model_dual, Min, boundaries' * u);

In [13]:
@constraint(model_dual, [i' * u for i in [cake_A_costs, cake_B_costs]] .>= profit);

In [14]:
model_dual

A JuMP Model
Minimization problem with:
Variables: 3
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: u

In [15]:
JuMP.optimize!(model_dual)

# Printing the optimal solutions obtained
println("Optimal Dual Solutions:")
println("u = ", JuMP.value.(u))

Optimal Dual Solutions:
u = [3.33333, 0.0, 10.0]


Can we get the dual solutions without building the dual problem using JuMP? **Yes**

In [16]:
JuMP.optimize!(model)

# Printing the optimal solutions obtained
println("Optimal Solutions:")
println("x = ", JuMP.value.(x))

Optimal Solutions:
x = [5.0, 22.5]


In [17]:
# Printing the optimal dual variables
println("Dual Variables:")
println("dual = ", JuMP.shadow_price.(con))

Dual Variables:
dual = [3.33333, 0.0, 10.0]


### Duality Theorem

Suppose the **primal problem** has (finite) optimal solution. Then the **dual problem** also has a (finite) optimal solution, and the corresponding values of the objective functions are equal. If the **primal** has no bounded optimum, then **dual** has no feasible solution. Symmetrically, if the **primal** has no feasible solution, then the **dual** has no bounded optimum.

### Exercise

A firm produces two goods $A$ and $B$. The firm earns a profit of $300$ from each unit of good $A$,
and $200$ from each unit of $B$. There are three stages of the production process. Good $A$ requires
six hours in production, then four hours in assembly, and finally five hours of packing. The corresponding numbers for $B$ are three, six, and five, respectively. The total number of hours available
for the three stages are $54$, $48$, and $50$, respectively.

1. Formulate and solve the $LP$ problem of maximizing profits subject to the given constraints;
2. Write down and solve the dual problem;
3. By how much would the optimal profit increase if the firm gets two hours more preparation time and one more packing time?

#### Solution

##### Point 1

In [18]:
profits = [300, 200];

good_A_prices = [6, 4, 5];
good_B_prices = [3, 6, 5];

hours_available = [54, 48, 50];

In [19]:
model = Model(with_optimizer(GLPK.Optimizer));

@variable(model, x[1:2] >= 0, base_name = "Good");

@objective(model, Max, profits' * x);

@constraint(model, (x[1] .* good_A_prices) + (x[2] .* good_B_prices) .<= hours_available);

In [20]:
model

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [21]:
JuMP.optimize!(model)

# Printing the optimal solutions obtained
println("Optimal Solutions:")
println("x = ", JuMP.value.(x))

Optimal Solutions:
x = [8.0, 2.0]


##### Point 2

In [22]:
model_dual = Model(with_optimizer(GLPK.Optimizer));

In [23]:
@variable(model_dual, u[1:3] >= 0, base_name = "Price");

In [24]:
@objective(model_dual, Min, hours_available' * u);

In [25]:
@constraint(model_dual, [i' * u for i in [good_A_prices,good_B_prices]] .>= profits);

In [26]:
model_dual

A JuMP Model
Minimization problem with:
Variables: 3
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: u

In [27]:
JuMP.optimize!(model_dual)

# Printing the optimal solutions obtained
println("Optimal Dual Solutions:")
println("u = ", JuMP.value.(u))

Optimal Dual Solutions:
u = [33.3333, 0.0, 20.0]


##### Point 3

In [28]:
Δπ = JuMP.value(u[1])*2 + JuMP.value(u[3])*1

86.66666666666667